In [1]:
import re
import numpy as np 
import matplotlib.pyplot as plt 
import whisper
import os
import pandas as pd
from Ort import Ort
from AudioFile import AudioFile

In [2]:
# Set the path to the audio files located in the files/recordings folder
AUDIO_FILES_PATH = os.getcwd() + "/files/recordings/"

# Metadata file path
METADATA_FILE_PATH = os.getcwd() + "/files/metadata/recordings_NL.txt"

# The path to the ORT files containing the transcriptions
ORT_TRANSCRIPTIONS_PATH = os.getcwd() + "/files/transcriptions/"

In [3]:
# Get the audio files for the DT subgroup (Native teenagers between 12 and 16 years old)
audio_files = AudioFile.get_audio_files_by_subgroup(METADATA_FILE_PATH, AUDIO_FILES_PATH, component="comp-q", subgroup="DT")

print("there are " + str(len(audio_files)) + " audio files present.")


File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000106.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000139.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000147.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000153.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000155.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000157.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000377.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000381.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000384.wav. Skipping...
File not found: /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn0005

In [4]:
# Create the method for running the model
def run_model(audio_file_path):
    print("Transcribing " + audio_file_path)
    model = whisper.load_model("tiny")
    return model.transcribe(audio_file_path, fp16=False)

In [5]:
results = {}

# Run the model on all audio files
for audio_file in audio_files:
    # Get the file name without the extension so we can find the corresponding ORT file
    file_name = os.path.splitext(os.path.basename(audio_file))[0]
    
    # Run the model on the audio file
    result = run_model(audio_file)

    # append the result to the results dictionary with the file name as key
    results[file_name] = result

Transcribing /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000096.wav
Transcribing /Users/hannatoenbreker/AAI/RTL-project/rtl/files/recordings/fn000098.wav


In [10]:
from datasets import load_metric
import string

class results_gradio:
    def error_rate():
        wer = load_metric("wer")
        cer = load_metric("cer")

        # Calculate the WER and CER for the results
        wer_results = []
        cer_results = []

        # Loop trough the results dictionary
        for result in results.items():
            # Set magic numbers
            FILE_NAME_INDEX = 0
            TRANSCRIPTION_INDEX = 1

            # Get the path to the ORT file
            ort_file_path = os.getcwd() + "/files/transcriptions/" + result[FILE_NAME_INDEX] + ".ort"

            # Converting the ort transcription to a string and transform it to lowercase
            target = [(Ort.to_string(ort_file_path)).lower()]

            # Define the prediction, convert it to lowercase and remove punctuation
            prediction = [result[TRANSCRIPTION_INDEX]['text'].lower().translate(str.maketrans("", "", string.punctuation))]

            # compute metrics
            wer_result = wer.compute(references=target, predictions=prediction)
            cer_result = cer.compute(references=target, predictions=prediction)

            wer_results.append(wer_result)
            cer_results.append(cer_result)

        return {'wer': np.mean(wer_results), 'cer': np.mean(cer_results)}



In [7]:
print("Gemiddelde WER: " + str(np.mean(wer_results)))
print("Gemiddelde CER: " + str(np.mean(cer_results)))

NameError: name 'wer_results' is not defined